# Creation of a contact sequence

In this second notebook, we will manually create a contact sequence from a predefined gait. Then, we will add some centroidal data to the contact sequence and export it. 

## Contact plan

In this section we will create a contact sequence for a bipedal robot, with a gait alterning double support phases and single support phases, for a simple walking motion. 

First, we need to create the first contact phase: a phase with both feet in contact with a flat floor at `z=0`. All the values here are taken for the robot Talos.

In [1]:
# Import the required lib
import numpy as np
from pinocchio import SE3
from multicontact_api import (
    ContactType,
    ContactModel,
    ContactPatch,
    ContactPhase,
    ContactSequence,
)

# Define the name of the contacts.
# As explained in the previous notebook, a good practice is to use the names of the frames as defined in the urdf
rf_name = "leg_right_sole_fix_joint"
lf_name = "leg_left_sole_fix_joint"

OFFSET_Y = (
    0.085  # the position along the y axis of the feet in the reference configuration
)

# Create a contact phase:
p0 = ContactPhase()

# Define the placement of each contact:
placement_rf = SE3.Identity()
placement_lf = SE3.Identity()
translation_rf = np.array([0, -OFFSET_Y, 0])
translation_lf = np.array([0, OFFSET_Y, 0])
placement_rf.translation = translation_rf
placement_lf.translation = translation_lf

# Add both contacts to the contact phase:
p0.addContact(rf_name, ContactPatch(placement_rf))
p0.addContact(lf_name, ContactPatch(placement_lf))

print("First phase: \n", p0)

First phase: 
 Contact phase defined for t \in [-1;-1]
Conecting (c0,dc0,ddc0,L0,dL0) = 
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
to        (c0,dc0,ddc0,L0,dL0) = 
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
Effectors in contact 2 : 
______________________________________________
Effector leg_right_sole_fix_joint contact patch:
Placement:
  R =
1 0 0
0 1 0
0 0 1
  p =      0 -0.085      0

ContactModel : ContactType: 0, mu: -1
Number of contact points: 1, positions: 
0
0
0


Has contact force trajectory : 0
Has contact normal force trajectory : 0
______________________________________________
Effector leg_left_sole_fix_joint contact patch:
Placement:
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.085     0

ContactModel : ContactType: 0, mu: -1
Number of contact points: 1, positions: 
0
0
0


Has contact force trajectory : 0
Has contact normal force trajectory : 0



As you can see in the print, a lot of data in this contact phase are undefined, we will fill this data.
Now we can create an empty contact sequence and set this phase as the first one of the sequence:

In [2]:
cs = ContactSequence()
cs.append(p0)
print("Current size of the sequence : ", cs.size())

Current size of the sequence :  1


Now we can add more phases to define the walking motion. The final contact plan will consist of 5 steps for each leg of 20cm forward (with the first and last step only 10cm), thus moving the robot of 1m forward. Let's create the first step, remenber that in our formulation there should only be one contact variation (creation OR removing) between each adjacent phases, one step is thus two contact phases: single support and double support.

In [3]:
# First, create a new phase where we break the right feet contact:
p1 = ContactPhase(p0)  # copy the previous double support phase
p1.removeContact(rf_name)

# Now, add it to the sequence:
cs.append(p1)

# Then, create the second double support phase by creating a new contact with the right foot:
placement_rf = SE3.Identity()
translation_rf[0] = 0.1  # move 10cm along the x axis
placement_rf.translation = translation_rf

p2 = ContactPhase(p1)  # copy the previous phase
p2.addContact(rf_name, ContactPatch(placement_rf))

# Now, add it to the sequence:
cs.append(p2)

# Lets print the result:
print("number of contact phases in the contact sequence : ", cs.size())

# first phase:
print(
    "# Right feet contact of phase 0: ",
    cs.contactPhases[0].isEffectorInContact(rf_name),
)
print(cs.contactPhases[0].contactPatch(rf_name).placement)
print(
    "# Left feet contact of phase 0: ", cs.contactPhases[0].isEffectorInContact(lf_name)
)
print(cs.contactPhases[0].contactPatch(lf_name).placement)
# second phase:
print(
    "# Right feet contact of phase 1: ",
    cs.contactPhases[1].isEffectorInContact(rf_name),
)
print(
    "# Left feet contact of phase 1: ", cs.contactPhases[1].isEffectorInContact(lf_name)
)
print(cs.contactPhases[1].contactPatch(lf_name).placement)
# Third phase:
print(
    "# Right feet contact of phase 2: ",
    cs.contactPhases[2].isEffectorInContact(rf_name),
)
print(cs.contactPhases[2].contactPatch(rf_name).placement)
print(
    "# Left feet contact of phase 2: ", cs.contactPhases[2].isEffectorInContact(lf_name)
)
print(cs.contactPhases[2].contactPatch(lf_name).placement)

number of contact phases in the contact sequence :  3
# Right feet contact of phase 0:  True
  R =
1 0 0
0 1 0
0 0 1
  p =      0 -0.085      0

# Left feet contact of phase 0:  True
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.085     0

# Right feet contact of phase 1:  False
# Left feet contact of phase 1:  True
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.085     0

# Right feet contact of phase 2:  True
  R =
1 0 0
0 1 0
0 0 1
  p =    0.1 -0.085      0

# Left feet contact of phase 2:  True
  R =
1 0 0
0 1 0
0 0 1
  p =     0 0.085     0



As expected we now have a contact sequence with 3 phases: a double support, a single support and a double support. The code above is quite verbose, fortunately several helper methods exist to achieve the same result easier. Lets create the second step with the left foot with this helpers:

In [4]:
# This method add a new contact phase to the sequence,
# copy the contacts from the previous phase exept the one specified
cs.breakContact(lf_name)

translation_lf[0] = 0.2  # move 20cm forward
placement_lf.translation = translation_lf

# This method add a new contact phase to the sequence,
# copy the contacts from the previous phase and add the one specified
cs.createContact(lf_name, ContactPatch(placement_lf))

print("Current contact sequence size: ", cs.size())

Current contact sequence size:  5


For the next steps, we will use another helper usefull for gaited motion. This method is used to "reposition" a contact and automatically add the intermediate contact phase with the broken contact. 

In [5]:
# First define the step length:
displacement = SE3.Identity()
displacement.translation = np.array([0.2, 0, 0])  # 20cm forward

for _ in range(4):
    cs.moveEffectorOf(rf_name, displacement)
    cs.moveEffectorOf(lf_name, displacement)

# add the last step of only 10cm to end the motion with both feet side by side:
displacement.translation = np.array([0.1, 0, 0])
cs.moveEffectorOf(rf_name, displacement)

print("Final contact sequence size: ", cs.size())
print(
    "Right foot position at the end of the motion: \n",
    cs.contactPhases[-1].contactPatch(rf_name).placement.translation,
)
print(
    "Left foot position at the end of the motion: \n",
    cs.contactPhases[-1].contactPatch(lf_name).placement.translation,
)

Final contact sequence size:  23
Right foot position at the end of the motion: 
 [ 1.    -0.085  0.   ]
Left foot position at the end of the motion: 
 [1.    0.085 0.   ]


At this point the contact sequence define a consistent contact plan, we can check this with the following method:

In [6]:
cs.haveConsistentContacts()

True

This method check that there is no discontinuities between the phases, and that there is always 1 contact variation between each phases.


### Additionnal data

#### Contact model

The Contact phases created do not specify any contact model (see the previous notebook for more information about this). We can check that the contact model are indeed not defined with the following code:


In [7]:
print("Friction coefficient defined for all contacts: ", cs.haveFriction())
print("Contact models defined for all contacts: ", cs.haveContactModelDefined())

Friction coefficient defined for all contacts:  False
Contact models defined for all contacts:  False


 We can create a contact model specific to the robot feet and assign it to all the contact patches of all phases with this code:

In [8]:
# Create a contact model with a friction coefficient of 0.5 and the PLANAR type
contact_model = ContactModel(0.5, ContactType.CONTACT_PLANAR)

# Define 4 contacts points at the corners of a rectangle:
contact_model.num_contact_points = 4

lx = 0.2 / 2.0  # half size of the feet along x axis
ly = 0.13 / 2.0  # half size of the feet along y axis

contact_points = np.zeros([3, 4])
contact_points[0, :] = [-lx, -lx, lx, lx]
contact_points[1, :] = [-ly, ly, -ly, ly]
contact_model.contact_points_positions = contact_points

# Now, add this model to all patches of all phases:

for phase in cs.contactPhases:
    for ee_name in phase.effectorsInContact():
        phase.contactPatch(ee_name).contact_model = contact_model


print("Friction coefficient defined for all contacts: ", cs.haveFriction())
print("Contact models defined for all contacts: ", cs.haveContactModelDefined())

Friction coefficient defined for all contacts:  True
Contact models defined for all contacts:  True


#### Phase duration

As explained in the previous notebook, a contact phase may be defined on a specific time interval. We can check if this is correctly defined for all the phases and that the timings are consistent with the following method:

In [9]:
cs.haveTimings()

False

The code below set the duration for all the phases, depending on the number of contacts of this phase. We are going to define long phase duration here because in the next section we are going to generate a quasi-static centroidal reference.

In [10]:
DURATION_SS = 2.0  # duration of the single support phases
DURATION_DS = 4.0  # duration of the double support phases

for i, phase in enumerate(cs.contactPhases):
    if i == 0:
        phase.timeInitial = 0.0
    else:
        # Set the initial time as the final time of the previous phase
        phase.timeInitial = cs.contactPhases[i - 1].timeFinal
    # set the duration of the phase based on the number of contacts
    if phase.numContacts() == 1:
        phase.duration = DURATION_SS
    elif phase.numContacts() == 2:
        phase.duration = DURATION_DS
    else:
        raise RuntimeError("Incorrect number of contacts for the phase " + str(i))

# Check that the timings are correctly set :
print("Contact sequence have consistent timings: ", cs.haveTimings())

Contact sequence have consistent timings:  True


## Centroidal data

Now that the contact sequence correctly define a contact plan, we are going to store centroidal data to the sequence: the center of mass position, velocity and acceleration trajectory. 
As this notebook is about the multicontact_api package and not about centroidal trajectory optimization, we are going to use really simple and quasi-static trajectory:

During the single support phases, the CoM is fixed above the center of the support polygon. During the double support phases, the CoM will go from the previous support polygon to the next one in a straight line (starting and ending with a null velocity and acceleration).

First, we need to compute the initial and final CoM position for each phases, the CoM velocity and acceleration are initialized to 0 by defaut so we do not need to modify it here.

In [11]:
# Define the CoM height:
COM_HEIGHT = 0.85


for i, phase in enumerate(cs.contactPhases):
    if i == 0:
        # Define the initial CoM position:
        cs.contactPhases[0].c_init = np.array([0, 0, COM_HEIGHT])
    elif phase.numContacts() == 1:
        # Single support phase: set the CoM position above the feet in contact:
        com = phase.contactPatch(phase.effectorsInContact()[0]).placement.translation
        com[2] += COM_HEIGHT
        phase.c_init = com
        # The CoM is not moving during single support phase, so we set the same position as the final point
        phase.c_final = com
        # Set the final point of the previous double support phase to be the same as this position
        cs.contactPhases[i - 1].c_final = com
    elif phase.numContacts() == 2:
        # Double support phase:
        # set the initial CoM position to be equal to the final position of the previous phase
        phase.c_init = cs.contactPhases[i - 1].c_final
    else:
        raise RuntimeError("Incorrect number of contacts for the phase " + str(i))

# For the final phase: set the final position between the feets:
com = (
    phase.contactPatch(rf_name).placement.translation
    + phase.contactPatch(lf_name).placement.translation
) / 2.0
com[2] += COM_HEIGHT
phase.c_final = com

print("Final CoM position: ", cs.contactPhases[-1].c_final)

Final CoM position:  [1.   0.   0.85]


We can check that the position for all the phases have been set and that the initial position of each phase match the final position of the previous phase like this:

In [12]:
cs.haveCentroidalValues()

True

Now we can generate trajectories for each phases. For the single support phases we will use Constant trajectories. For the double support phases we will use fifth order polynomials that connect the initial and final position and have a null initial and final velocity and acceleration. 

As explained in the previous notebook, this trajectories must be represented with objects from the `NDCurves` package.


In [15]:
from ndcurves import polynomial

for phase in cs.contactPhases:
    if phase.numContacts() == 1:
        # Single support phase: build a constant trajectory at the CoM position:
        phase.c_t = polynomial(
            phase.c_init, phase.timeInitial, phase.timeFinal
        )  # Build a degree 0 polynomial curve
        # Compute the derivate of this curve and store it in the phase
        phase.dc_t = phase.c_t.compute_derivate(1)
        phase.ddc_t = phase.dc_t.compute_derivate(1)
    elif phase.numContacts() == 2:
        # Double support phase: build a minJerk trajectory (5th degree) between the initial and final CoM position
        phase.c_t = polynomial.MinimumJerk(
            phase.c_init, phase.c_final, phase.timeInitial, phase.timeFinal
        )
        phase.dc_t = phase.c_t.compute_derivate(1)
        phase.ddc_t = phase.dc_t.compute_derivate(1)
    else:
        raise RuntimeError("Incorrect number of contacts.")


# Check that all the phases have CoM trajectories:
print("Contact sequence have consistent CoM trajectories: ", cs.haveCOMtrajectories())

Contact sequence have consistent CoM trajectories:  True


## Conclusion

In this notebook we saw how to manually build a `ContactSequence` from scratch. Adding several phases with contact placement and time interval. Then adding CoM trajectories for each phases. 

In the next notebook we will se how to use the data inside a contact sequence, plot it and display it on a 3D viewer. 

The last code below show how to export the ContactSequence build in this notebook to a file. 

In [16]:
# Serialize the contact sequence
cs.saveAsBinary("notebook2.cs")